Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

このアサインメントのゴールは、徐々にトレーニングをしてより精度があるモデルをTensorFlowで作ります

In [11]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

最初に "1_notmnist.ipynb" で作成したデータをロードします。

In [12]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

shapeをモデルに適合する形に変換します。
- データを、28x28をフラットの784(28*28)にする
- ラベルを、One-Hotエンコーディングにする（0 => [1, 0,  0, ..], 1 => [0, 1, 0, ...])

In [13]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

---

勾配降下法(Gradient Descent)を使って、多項ロジスティック回帰でトレーニングをします。

TensorFlowは次のように動きます。
* 最初に、入力、変数、操作(operations)などの計算(computation)を定義します。計算グラフ上でノードとして作成されます。`with graph.as_default()`ブロック内で定義します。
* 次に、`session.run()`を使って、このグラフ上で操作を実行します。


では、TensorFlow内にデータをロードし、トレーニングに適切な計算グラフを構築します

In [14]:
# ロジスティック回帰の宣言（Optimizer: Gradient Decent)

# 勾配降下法(Gradietn Decent)では、多くのデータを渡すと計算量が多くなってしまうので、
# サブセットとして10,000件のみでトレーニングデータでとトレーニングするようにします
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # 入力
    # トレーニング、バリデーション、テストデータを定数(constant)として読み込む
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # 変数
    # トレーニングを行うパラメータです。
    # 重み(weights)は、ポアソン切断正規分布に応じたランダム値で初期化します
    # バイアス(biases)は、0で初期化します
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # トレーニングの計算 (Training computation)
    # 入力と重み(weight)を掛け、バイアスを足します。
    # Softmaxとクロスエントロピーを計算します（TensorFlowでは、よく使われるので１つの操作になっています）
    # コスト(loss)は、各トレーニングのクロスエントロピーの平均をとります。
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # 最適化 (Optimizer)
    # 勾配降下法(gradient descent)を使ってコスト(loss)を最小化します
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # トレーニング、バリデーション、テストデータのそれぞれで予測をします
    # トレーニングの一部ではないですが、トレーニングの精度を測るのに必要です。
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

計算を実行し、イテレートしましょう

In [15]:
# ロジスティック回帰（実行）
# Last Training accuracy: 79.4%, Validation accuracy: 74.4%, Test accuracy: 82.0%

num_steps = 801

def accuracy(predictions, labels):
    """予測(predictions)と正解ラベル(labels)で精度を返す"""
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
                  / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # 変数を初期化させるため１度だけ実行するオペレーションです。
    # 重みをランダムな値で、バイアスを0で初期化します
    tf.initialize_all_variables().run()
    print('変数を初期化しました')

    for step in range(num_steps):
        # 計算を実行します。run()により最適化(optimizer)、コスト、トレーニングの予測を実行します。
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
                        predictions, train_labels[:train_subset, :]))

    # .eval() は run()のように実行できます。しかし、numpy配列を1つしか返すことができません
    # これは、グラフ内で再計算をします
    print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

変数を初期化しました
Loss at step 0: 18.621544
Training accuracy: 10.5%
Loss at step 100: 2.265072
Training accuracy: 72.7%
Loss at step 200: 1.810284
Training accuracy: 75.4%
Loss at step 300: 1.569070
Training accuracy: 76.6%
Loss at step 400: 1.409667
Training accuracy: 77.6%
Loss at step 500: 1.291587
Training accuracy: 78.4%
Loss at step 600: 1.198748
Training accuracy: 78.8%
Loss at step 700: 1.123192
Training accuracy: 79.5%
Loss at step 800: 1.060325
Training accuracy: 80.0%
Validation accuracy: 74.5%
Test accuracy: 82.5%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

---

さあ、より早く学習ができる、確率的勾配降下法(SGD: Stochastic Gradient Descent)でトレーニングするようにしましょう。

グラフは似ていますが、constantノードを、palceholderノードに変えています。placeholderノードは`session.run()`時に実際のデータを渡すことができるようになります。

In [16]:
# ロジスティック回帰の宣言（Optimizer: SGD)
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # 入力データ (Input data)
    # トレーニングデータは、placeholderで宣言し、トレーニング実行時に
    # ミニバッチとしてデータを渡せるようにします
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # 変数 (Valriables)
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # トレーニングの計算 (Training computation)
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # 最適化(Optimizer.)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # トレーニング、バリデーション、テストデータに対しての予測
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [17]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("変数を初期化しました")

    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        # ミニバッチの作成
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

変数を初期化しました
Minibatch loss at step 0: 17.412159
Minibatch accuracy: 12.5%
Validation accuracy: 9.7%
Minibatch loss at step 500: 1.268969
Minibatch accuracy: 83.6%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 1.470032
Minibatch accuracy: 76.6%
Validation accuracy: 76.8%
Minibatch loss at step 1500: 0.754533
Minibatch accuracy: 82.0%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 0.881235
Minibatch accuracy: 82.8%
Validation accuracy: 77.2%
Minibatch loss at step 2500: 1.145980
Minibatch accuracy: 75.0%
Validation accuracy: 77.8%
Minibatch loss at step 3000: 0.933925
Minibatch accuracy: 78.1%
Validation accuracy: 79.2%
Test accuracy: 86.2%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

SGDのロジスティック回帰を、ニューラルネットワークに変えてみてください。隠れ層は、1024ノードで1層、アクティベーションは ReLU (`nn.relu()`)を使います。
このモデルにより、バリデーションとテストの精度が向上するでしょう。

---

In [18]:
# ニューラルネットワーク
# input:      [batch_size, 784(28x28)]
# hidden1: weights(784x1024) + bias(1024) => relu
# output:   weights(1024x10) + bias(10) => softmax, cross_entropy
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

    # 入力データ (Input data)
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # 変数 (Valriables)
    hidden1_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
    hidden1_biases = tf.Variable(tf.zeros([hidden_size]))
    output_weights = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
    output_biases = tf.Variable(tf.zeros([num_labels]))

    # トレーニングの計算 (Training computation)
    # 隠れ層の計算
    hidden1_Wx_plus_b = tf.matmul(tf_train_dataset, hidden1_weights) + hidden1_biases
    hidden1_activation = tf.nn.relu(hidden1_Wx_plus_b)
    
    # 出力層の計算
    logits = tf.matmul(hidden1_activation, output_weights) + output_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # 最適化(Optimizer.)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # トレーニング、バリデーション、テストデータに対しての予測
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden1_weights) + hidden1_biases), output_weights) + output_biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden1_weights) + hidden1_biases), output_weights) + output_biases)

In [19]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("変数を初期化しました")

    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        # ミニバッチの作成
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

変数を初期化しました
Minibatch loss at step 0: 334.850464
Minibatch accuracy: 9.4%
Validation accuracy: 32.2%
Minibatch loss at step 500: 19.017977
Minibatch accuracy: 80.5%
Validation accuracy: 80.1%
Minibatch loss at step 1000: 8.198958
Minibatch accuracy: 75.0%
Validation accuracy: 81.8%
Minibatch loss at step 1500: 5.981525
Minibatch accuracy: 85.9%
Validation accuracy: 78.9%
Minibatch loss at step 2000: 3.409707
Minibatch accuracy: 86.7%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 6.903316
Minibatch accuracy: 81.2%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 1.719259
Minibatch accuracy: 83.6%
Validation accuracy: 81.9%
Test accuracy: 89.2%
